In [ ]:
!pip install scikit-learn xgboost tensorflow


In [2]:
import cuml
import sklearn
import tensorflow

cuml_version = cuml.__version__
sklearn_version = sklearn.__version__
tensorflow_version = tensorflow.__version__

print(f"cuML version: {cuml_version}")
print(f"sklearn version: {sklearn_version}")
print(f"tensorflow version: {tensorflow_version}")


cuML version: 25.10.00
sklearn version: 1.6.1
tensorflow version: 2.19.0


### 04-01-01. Linear Regression

In [5]:
%%time

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score


# 가상 데이터 생성 (scikit-learn 라이브러리 사용)
X, y = make_classification(n_classes=2,
                           n_features=100,
                           n_samples=1000000,
                           random_state=0)

# 데이터셋을 학습(75%) 및 테스트(25%)로 분할
# train_test_split의 test_size 기본값은 0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# scikit-learn 선형 회귀 모델 생성
model = LinearRegression()

# 모델 학습: 학습 데이터(X_train, y_train)를 사용하여 모델을 학습시킴
model.fit(X_train, y_train)

# 학습한 모델의 예측: 테스트 데이터(X_test)를 사용하여 확률 예측
y_pred = model.predict(X_test)

# accuracy_score를 사용하기 위해 예측 결과를 이진 분류로 변환
# 확률이 0.5 이상이면 1, 미만이면 0으로 처리
y_pred_binary = (y_pred >= 0.5).astype(int)

# 정확도 측정 : 실제 값(y_test)과 이진 분류된 예측 값(y_pred_binary)을 비교하여 정확도 계산
sklearn_accuracy = accuracy_score(y_test, y_pred_binary)

# 정확도 출력
print('scikit-learn Linear Regression 정확도:', sklearn_accuracy)


scikit-learn Linear Regression 정확도: 0.905028
CPU times: user 20.1 s, sys: 1.28 s, total: 21.4 s
Wall time: 17.2 s


In [6]:
%%time

from cuml.datasets.classification import make_classification
from cuml.model_selection import train_test_split
from cuml.linear_model import LinearRegression
from cuml.metrics import accuracy_score
import cupy as cp


# 가상 데이터 생성 (cuML 라이브러리 사용, GPU에 데이터 생성)
X, y = make_classification(n_classes=2,
                           n_features=100,
                           n_samples=1000000,
                           random_state=0)

# 데이터셋을 학습(75%) 및 테스트(25%)로 분할
# train_test_split의 test_size 기본값은 0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# cuML 선형 회귀 모델 생성
# copy_X=True: 입력 데이터 X의 복사본을 생성하여 원본 데이터의 변형을 방지
# copy_X=False: 입력 데이터 X의 복사본을 생성하지 않고 메모리 절약 가능하나 원본 데이터가 변경될 수 있음
model = LinearRegression(copy_X=True)

# 모델 학습: 학습 데이터(X_train, y_train)를 사용하여 모델을 학습시킴
model.fit(X_train, y_train)

# 학습한 모델의 예측: 테스트 데이터(X_test)를 사용하여 확률 예측
y_pred = model.predict(X_test)

# accuracy_score를 사용하기 위해 예측 결과를 이진 분류로 변환
# 확률이 0.5 이상이면 1, 미만이면 0으로 처리
y_pred_binary = (y_pred >= 0.5).astype(cp.int32)

# 정확도 측정 : 실제 값(y_test)과 이진 분류된 예측 값(y_pred_binary)을 비교하여 정확도 계산
cuml_accuracy = accuracy_score(y_test, y_pred_binary)

# 정확도 출력
print('cuML Linear Regression 정확도:', cuml_accuracy)


cuML Linear Regression 정확도: 0.99512
CPU times: user 328 ms, sys: 19.8 ms, total: 347 ms
Wall time: 343 ms


### 04-01-02. k-NN


In [7]:
%%time

from tensorflow.keras.datasets import mnist

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


# MNIST 데이터셋 불러오기
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 데이터를 1차원 벡터로 변환 (각 이미지가 28x28에서 784로 변환됨)
X_train = X_train.reshape(X_train.shape[0], -1)  # 크기 (60,000, 784)
X_test = X_test.reshape(X_test.shape[0], -1)  # 크기 (10,000, 784)

# scikit-learn k-NN 모델 초기화 (n_neighbors 기본값은 5)
knn_model = KNeighborsClassifier(n_neighbors=3)

# 모델 학습
knn_model.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = knn_model.predict(X_test)

# 정확도 측정
sklearn_accuracy = accuracy_score(y_test, y_pred)

# 정확도 출력
print('scikit-learn k-NN Classifier 정확도:', sklearn_accuracy)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
scikit-learn k-NN Classifier 정확도: 0.9705
CPU times: user 33.1 s, sys: 2.93 s, total: 36 s
Wall time: 32.4 s


In [8]:
%%time
from tensorflow.keras.datasets import mnist

import cudf
import cuml
from cuml.neighbors import KNeighborsClassifier
from cuml.metrics import accuracy_score


# MNIST 데이터셋 불러오기
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 데이터를 1차원 벡터로 변환 (각 이미지가 28x28에서 784로 변환됨)
X_train = X_train.reshape(X_train.shape[0], -1) # (60,000, 784)
X_test = X_test.reshape(X_test.shape[0], -1) # (10,000, 784)

# cuDF DataFrame으로 변환하여 GPU 메모리에 올리기
X_train_cudf = cudf.DataFrame.from_records(X_train)
X_test_cudf = cudf.DataFrame.from_records(X_test)
y_train_cudf = cudf.Series(y_train)
y_test_cudf = cudf.Series(y_test)

# cuML k-NN 모델 초기화 (n_neighbors 기본값은 5)
knn_model = KNeighborsClassifier(n_neighbors=3)

# 모델 학습
knn_model.fit(X_train_cudf, y_train_cudf)

# 테스트 데이터 예측
y_pred_cudf = knn_model.predict(X_test_cudf)

# 정확도 측정
cuml_accuracy = accuracy_score(y_test_cudf, y_pred_cudf)

# 정확도 출력
print('cuML k-NN Classifier 정확도:', cuml_accuracy)


cuML k-NN Classifier 정확도: 0.9705
CPU times: user 3.51 s, sys: 77.3 ms, total: 3.59 s
Wall time: 4.32 s


### 04-01-03. t-SNE


In [ ]:
%%time

from tensorflow.keras.datasets import mnist

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


# MNIST 데이터셋 불러오기
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 데이터를 1차원 벡터로 변환 (각 이미지가 28x28에서 784로 변환됨)
X_train = X_train.reshape(X_train.shape[0], -1)  # (60,000, 784)
X_test = X_test.reshape(X_test.shape[0], -1)  # (10,000, 784)

# scikit-learn t-SNE 모델 초기화 및 피팅
tsne = TSNE(n_components=2, perplexity=30, random_state=0)
X_tsne = tsne.fit_transform(X_train)

plt.figure(figsize=(10, 8)) # 그래프 크기 설정
scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_train, cmap='tab10', alpha=0.5) # 결과 시각화
plt.title('t-SNE visualization of MNIST dataset (scikit-learn)')
plt.xlabel('t-SNE component 1') # x축 라벨 설정
plt.ylabel('t-SNE component 2') # y축 라벨 설정
plt.legend(*scatter.legend_elements(), title="Classes") # 범주 추가
plt.colorbar(scatter) # 색상 막대 추가
plt.show() # 그래프 출력


In [ ]:
%%time

from tensorflow.keras.datasets import mnist

import cudf
import cuml
from cuml.manifold import TSNE
import matplotlib.pyplot as plt


# MNIST 데이터셋 불러오기
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 데이터를 1차원 벡터로 변환 (각 이미지가 28x28에서 784로 변환됨)
X_train = X_train.reshape(X_train.shape[0], -1)  # (60,000, 784)
X_test = X_test.reshape(X_test.shape[0], -1)  # (10,000, 784)

# cudf DataFrame으로 변환하고 GPU 메모리로 이동
X_train_cudf = cudf.DataFrame(X_train)
y_train_cudf = cudf.Series(y_train)

# cuML t-SNE 모델 초기화 및 피팅
# method='barnes_hut' : scikit-learn과 동일한 method 사용
tsne = TSNE(n_components=2, perplexity=30, method='barnes_hut', random_state=0)
X_tsne_cudf = tsne.fit_transform(X_train_cudf)

# cudf.DataFrame을 numpy 배열로 변환하여 시각화
X_tsne_np = X_tsne_cudf.to_numpy()

plt.figure(figsize=(10, 8)) # 그래프 크기 설정
scatter = plt.scatter(X_tsne_np[:, 0], X_tsne_np[:, 1], c=y_train, cmap='tab10', alpha=0.5) # 결과 시각화
plt.title('t-SNE visualization of MNIST dataset (cuML)')
plt.xlabel('t-SNE component 1') # x축 라벨 설정
plt.ylabel('t-SNE component 2') # y축 라벨 설정
plt.legend(*scatter.legend_elements(), title="Classes") # 범주 추가
plt.colorbar(scatter) # 색상 막대 추가
plt.show() # 그래프 출력


### 04-01-04. DBSCAN


In [ ]:
%%time

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_moons
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


# scikit-learn make_moons 데이터 생성
X, y = make_moons(n_samples=150000, noise=0.1, random_state=0)

# scikit-learn DBSCAN 모델 초기화
dbscan = DBSCAN(eps=0.25, min_samples=1000)

# Numpy 배열 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# X_scaled에 DBSCAN을 적용하고 클러스터 레이블을 반환
clusters = dbscan.fit_predict(X_scaled)

# 시각화: 클러스터를 색상으로 구분, 노이즈는 -1로 구분
plt.figure(figsize=(10, 8))
unique_labels = np.unique(clusters)
for label in unique_labels:
    if label == -1:
        # 노이즈 포인트 (클러스터에 속하지 않은 포인트는 검은색으로 표시)
        plt.scatter(X_scaled[clusters == label, 0], X_scaled[clusters == label, 1], color='k', s=2, label='Noise')
    else:
        plt.scatter(X_scaled[clusters == label, 0], X_scaled[clusters == label, 1], s=2, label=f'Cluster {label}')

plt.axis('off')  # 축과 박스 제거
plt.title('DBSCAN Clustering of Moons Dataset (scikit-learn)')
plt.legend() # 범주 표시
plt.show()


In [ ]:
%%time

import numpy as np
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

import cudf
import cuml
from cuml.cluster import DBSCAN
from cuml.preprocessing import StandardScaler


# scikit-learn make_moons 데이터 생성
X, y = make_moons(n_samples=150000, noise=0.1, random_state=0)

# Numpy 배열 스케일링 (데이터가 GPU 메모리로 이동)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# cuML DBSCAN 모델 초기화
dbscan = DBSCAN(eps=0.25, min_samples=1000)

# X_scaled에 DBSCAN을 적용하고 클러스터 레이블을 반환
clusters = dbscan.fit_predict(X_scaled)

# 시각화: 클러스터를 색상으로 구분, 노이즈는 -1로 구분
plt.figure(figsize=(10, 8))
unique_labels = np.unique(clusters)
for label in unique_labels:
    if label == -1:
        # 노이즈 포인트 (클러스터에 속하지 않은 포인트는 검은색으로 표시)
        plt.scatter(X_scaled[clusters == label, 0], X_scaled[clusters == label, 1], color='k', s=2, label='Noise')
    else:
        plt.scatter(X_scaled[clusters == label, 0], X_scaled[clusters == label, 1], s=2, label=f'Cluster {label}')

plt.axis('off')  # 축과 박스 제거
plt.title('DBSCAN Clustering of Moons Dataset (cuML)')
plt.legend() # 범주 표시
plt.show()


### 04-01-05. Random Forest

In [ ]:
%%time

from tensorflow.keras.datasets import fashion_mnist
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Fashion MNIST 데이터 로드
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# 데이터 전처리 (평탄화 및 스케일링)
X_train = X_train.reshape(X_train.shape[0], -1).astype('float32') / 255.0
X_test = X_test.reshape(X_test.shape[0], -1).astype('float32') / 255.0

# scikit-learn RandomForestClassifier 초기화 및 학습
rf_model = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42)
rf_model.fit(X_train, y_train)

# 테스트 데이터 예측
y_test_pred = rf_model.predict(X_test)

# 정확도 측정
sklearn_accuracy = accuracy_score(y_test, y_test_pred)

# 정확도 출력
print('scikit-learn Random Forest Classifier 정확도:', sklearn_accuracy)


In [ ]:
%%time

from tensorflow.keras.datasets import fashion_mnist
from cuml.ensemble import RandomForestClassifier as cuMLRandomForestClassifier
from cuml.preprocessing import StandardScaler
from cuml.model_selection import train_test_split
from cuml.metrics import accuracy_score


# Fashion MNIST 데이터 로드
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# 데이터 전처리 (평탄화 및 스케일링)
X_train = X_train.reshape(X_train.shape[0], -1).astype('float32') / 255.0
X_test = X_test.reshape(X_test.shape[0], -1).astype('float32') / 255.0

# cuML RandomForestClassifier 초기화 및 학습
# n_streams=1 : 재현 가능성 보장
cuml_model = cuMLRandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, n_streams=1)
cuml_model.fit(X_train, y_train)

# 테스트 데이터 예측
y_test_pred = cuml_model.predict(X_test)

# 정확도 측정
cuml_accuracy = accuracy_score(y_test, y_test_pred)

# 정확도 출력
print('cuML Random Forest Classifier 정확도:', cuml_accuracy)
